In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch 
import torchvision 
import os  
import cv2
from PIL import Image
import numpy as np 

In [3]:
# direccion del dataset
dataset_flores = "/content/drive/MyDrive/IA2/dataset_flores/"


In [4]:
# leemos los labels que se toman de los nombres de las carpetas
clases = os.listdir(dataset_flores)
clases 

['lirio', 'Hibisco', 'Girasoles', 'rosas', 'FlorRita', 'tagetes', 'claveles']

In [5]:
label = {'Hibisco':0, 'tagetes':1, 'FlorRita':2, 'lirio':3, 'rosas':4, 'claveles':5, 'Girasoles':6}

In [6]:
# agregamos los labels y imagenes e un arreglo 
imgs, labels = [], []
img_resize = 32
for dir in clases:
  dir_img = os.listdir(dataset_flores + '/' + dir)
  #print(dir_img)
  for i in dir_img:
    path = dataset_flores + dir + '/' + i
    #print(path)
    imges = Image.open(path)
    #print(imges)
    #resize_img = cv2.resize(imges, (img_resize, img_resize))
    resize_img = imges.resize((img_resize, img_resize))
    imgs.append(np.array(resize_img))
    labels.append(label[dir])


el dataset que estamos usando tiene un total de 787 imagesnes

In [7]:
flores = np.array(imgs)
flores.shape

(7774, 32, 32, 3)

In [8]:
from sklearn.model_selection import train_test_split
train_sen, test_sen, train_labels, test_labels = train_test_split(imgs, labels, test_size= 0.2, stratify=labels)
len(train_sen), len(test_sen)       

(6219, 1555)

In [44]:
from skimage import io
device = "cuda" if torch.cuda.is_available() else "cpu"

class Dataset(torch.utils.data.Dataset):
  def __init__(self, X, y):
    self.X = X 
    self.y = y
  def __len__(self):
    return len(self.X)

  def __getitem__(self, ix):
    # cargar la imágen
    img = io.imread(self.X[ix])
    # aplicar transformaciones
    if self.trans:
      img = self.trans(image=img)["image"]
    return torch.from_numpy(img / 255.).float().permute(2,0,1), torch.tensor(self.y[ix])

In [48]:
dataset = {
    'train': Dataset(train_sen, train_labels), 
    'test': Dataset(test_sen, test_labels)
}
Xtrain = dataset['train']
XText = dataset['test']
Xtrain

In [51]:
dataloader = {
    'train': torch.utils.data.DataLoader(dataset['train'], batch_size=20, shuffle=True, pin_memory=True), 
    'test': torch.utils.data.DataLoader(dataset['test'], batch_size=20, shuffle=False)
}
imgs, labels = next(iter(dataloader['train']))
imgs.shape

OSError: ignored

In [40]:
def block(c_in, c_out, k=3, p=1, s=1, pk=2, ps=2):
    return torch.nn.Sequential(
        torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(pk, stride=ps)
    )

def block2(c_in, c_out):
    return torch.nn.Sequential(
        torch.nn.Linear(c_in, c_out),
        torch.nn.ReLU()
    )

class CNN(torch.nn.Module):
  def __init__(self, n_channels=1, n_outputs=10):
    super().__init__()
    self.conv1 = block(n_channels, 64)
    self.conv2 = block(64, 128)
    self.fc = torch.nn.Linear(128*7*7, n_outputs)

  def forward(self, x):
    print("Dimensiones:")
    print("Entrada: ", x.shape)
    x = self.conv1(x)
    print("conv1: ", x.shape)
    x = self.conv2(x)
    print("conv2: ", x.shape)
    x = x.view(x.shape[0], -1)
    print("pre fc: ", x.shape)
    x = self.fc(x)
    print("Salida: ", x.shape)
    return x

In [37]:
model = CNN()
output = model(torch.randn(64, 1, 28, 28))

Dimensiones:
Entrada:  torch.Size([64, 1, 28, 28])
conv1:  torch.Size([64, 64, 14, 14])
conv2:  torch.Size([64, 128, 7, 7])
pre fc:  torch.Size([64, 6272])
Salida:  torch.Size([64, 10])


In [26]:
class CNN(torch.nn.Module):
  def __init__(self, n_channels=1, n_outputs=10):
    super().__init__()
    self.conv1 = block(n_channels, 64)
    self.conv2 = block(64, 128)
    self.fc = torch.nn.Linear(128*7*7, n_outputs)

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = x.view(x.shape[0], -1)
    x = self.fc(x)
    return x

In [27]:
from tqdm import tqdm
import numpy as np

def fit(model, dataloader, epochs=5):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        bar = tqdm(dataloader['test'])
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")

In [28]:
model = CNN()
fit(model, dataloader)

  0%|          | 0/311 [00:00<?, ?it/s]


OSError: ignored